In [1]:
import glob
import clip
import os
from torch import nn
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as nnf
import sys
from typing import Tuple, List, Union, Optional
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
# from google.colab import files
import skimage.io as io
import PIL.Image
from IPython.display import Image
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import requests

from clip_prefix_captioning import MLP, ClipCaptionModel, ClipCaptionPrefix, generate_beam, generate2
from nltk.sentiment import SentimentIntensityAnalyzer

In [2]:
D = torch.device
CPU = torch.device('cpu')

is_gpu = True

def get_device(device_id: int) -> D:
    if not torch.cuda.is_available():
        return CPU
    device_id = min(torch.cuda.device_count() - 1, device_id)
    return torch.device(f'cuda:{device_id}')


CUDA = get_device

In [3]:
current_directory = os.getcwd()
save_path = os.path.join(os.path.dirname(current_directory), "pretrained_models")
os.makedirs(save_path, exist_ok=True)
model_path = os.path.join(save_path, 'model_weights.pt')

In [4]:
device = CUDA(0) if is_gpu else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [5]:
prefix_length = 10

model = ClipCaptionModel(prefix_length)

model.load_state_dict(torch.load(model_path, map_location=CPU)) 

model = model.eval() 
device = CUDA(0) if is_gpu else "cpu"
model = model.to(device)

In [6]:
def generate_caption(img_file, display_img=False, use_beam_search=False, return_all_beam=False, return_clip_emb=False):
    image = io.imread(img_file)
    pil_image = PIL.Image.fromarray(image)
    if display_img:
        display(pil_image.resize((512, 512)))
    image = preprocess(pil_image).unsqueeze(0).to(device)
    with torch.no_grad():
        prefix = clip_model.encode_image(image).to(device, dtype=torch.float32)
        prefix_embed = model.clip_project(prefix).reshape(1, prefix_length, -1)
    if use_beam_search:
        if return_all_beam:
            generated_text_prefix = generate_beam(model, tokenizer, embed=prefix_embed)
        else:
            generated_text_prefix = generate_beam(model, tokenizer, embed=prefix_embed)[0]
    else:
        generated_text_prefix = generate2(model, tokenizer, embed=prefix_embed)

    if return_clip_emb:
        return generated_text_prefix, prefix
    else:
        return generated_text_prefix

In [7]:
def clip_encode_caption(caption):
    with torch.no_grad():
        caption_tokenized = clip.tokenize(caption, truncate=True).to(device)
        clip_caption_rep = clip_model.encode_text(caption_tokenized).to(device, dtype=torch.float32)
    return clip_caption_rep

In [8]:
def choose_best_caption(captions, img_clip_emb):
    sia = SentimentIntensityAnalyzer()

    captions_clip_embs = [clip_encode_caption(caption).squeeze(dim=0).cpu().numpy() for caption in captions]
    emb_mtx = np.stack(np.array([img_clip_emb] + captions_clip_embs))

    semantic_sim_scores = cosine_similarity(emb_mtx)[0,1:]
    polairty_scores = np.array([sia.polarity_scores(caption)['neu'] for caption in captions])
    length_scores = np.array([len(caption.split(' ')) for caption in captions])
    winner = np.argmax(semantic_sim_scores + polairty_scores + length_scores)

    return captions[winner], captions_clip_embs[winner], polairty_scores[winner]

In [21]:
def find_most_sim_emb(input_emb_idx, sim_mtx):
    sim_mtx[input_emb_idx,input_emb_idx] = -1
    idx_most_sim = sim_mtx[input_emb_idx].argmax()
    return idx_most_sim

# Download Unsplash Dataset

In [9]:
data_files_path = os.path.join(os.path.dirname(current_directory), "data", "unsplash-research-dataset-lite-latest")
documents = ['photos', 'keywords', 'collections', 'conversions', 'colors']
datasets = {}

for doc in documents:
    files = glob.glob(data_files_path + '/' + doc + ".tsv*")
    print(files)
    subsets = []
    for filename in files:
        df = pd.read_csv(filename, sep='\t', header=0)
        subsets.append(df)

    datasets[doc] = pd.concat(subsets, axis=0, ignore_index=True)

['/Users/kgaiger/PycharmProjects/clip_new/CLIP_prefix_caption-1/data/unsplash-research-dataset-lite-latest/photos.tsv000']
['/Users/kgaiger/PycharmProjects/clip_new/CLIP_prefix_caption-1/data/unsplash-research-dataset-lite-latest/keywords.tsv000']
['/Users/kgaiger/PycharmProjects/clip_new/CLIP_prefix_caption-1/data/unsplash-research-dataset-lite-latest/collections.tsv000']
['/Users/kgaiger/PycharmProjects/clip_new/CLIP_prefix_caption-1/data/unsplash-research-dataset-lite-latest/conversions.tsv000']
['/Users/kgaiger/PycharmProjects/clip_new/CLIP_prefix_caption-1/data/unsplash-research-dataset-lite-latest/colors.tsv000']


In [10]:
to_download = datasets['photos'].head(10000).index

In [11]:
imgs_path = os.path.join(os.path.dirname(current_directory), "data", "unsplash-research-dataset-lite-latest", "images")
if not os.path.exists(imgs_path):
    os.makedirs(imgs_path)

In [ ]:
for index in tqdm(to_download):
    try:
        response = requests.get(datasets['photos'].loc[index,'photo_image_url'])
        file = open(f'{imgs_path}/{index}.png', "wb")
        file.write(response.content)
        file.close()
    except:
        print(f'index {index} failed!')

# Calculate CLIP and Style embeddings

In [30]:
tested = datasets['photos'].loc[to_download]

In [31]:
style_embs = {}
clip_embs = {}
caption_embs = {}

for img_file in os.listdir(imgs_path)[:10]:
    try:
        captions, img_clip_emb = generate_caption(f'{imgs_path}/{img_file}', use_beam_search=True, return_all_beam=True, return_clip_emb=True)
        img_clip_emb = img_clip_emb.squeeze(dim=0).cpu().numpy()
        caption, caption_clip_emb, caption_neu_score = choose_best_caption(captions, img_clip_emb)
        style = img_clip_emb - caption_clip_emb
        index = int(img_file.replace('.png',''))
        style_embs[index] = style
        clip_embs[index] = img_clip_emb
        caption_embs[index] = caption_clip_emb
        tested.loc[index, 'caption'] = caption
        tested.loc[index, 'neu_score'] = caption_neu_score
    except:
        continue

In [33]:
tested = tested.reset_index()

In [34]:
tested['style_emb'] = tested['index'].map(style_embs)
tested['clip_emb'] = tested['index'].map(clip_embs)
tested['caption_emb'] = tested['index'].map(caption_embs)

In [35]:
tested = tested[tested['style_emb'].notnull()]

In [36]:
tested = tested.reset_index(drop=True)

In [42]:
style_sims = cosine_similarity(np.stack(np.array(tested['style_emb'].tolist())))
tested['most_sim_style'] =  tested.apply(lambda row: tested.loc[find_most_sim_emb(row.name, style_sims),
                                                                'photo_url'], axis=1)
clip_sims = cosine_similarity(np.stack(np.array(tested['clip_emb'].tolist())))
tested['most_sim_clip'] = tested.apply(lambda row: tested.loc[find_most_sim_emb(row.name, clip_sims),
                                                                'photo_url'], axis=1)

In [43]:
tested.loc[0]

level_0                                                                         348
index                                                                           348
photo_id                                                                BHe9mQ6_gbg
photo_url                                   https://unsplash.com/photos/BHe9mQ6_gbg
photo_image_url                   https://images.unsplash.com/photo-157798121942...
photo_submitted_at                                       2020-01-02 16:09:18.465787
photo_featured                                                                    t
photo_width                                                                    4024
photo_height                                                                   6048
photo_aspect_ratio                                                             0.67
photo_description                                     Praděd Lookout tower at night
photographer_username                                                       